# Kennissessie Neurale Netwerken

# Deel 1
Deze sessie gaan we neurale netwerken trainen om simpele images te classificeren van de CIFAR-10 dataset. 

**Stap 1: data inladen**

De data bestaat uit 3 delen, train, validatie en test set. Er is veel herhaling in deze code. Splits dit op in leesbare en herbruikbare code.

In [1]:
import numpy as np
import os
import tarfile
from urllib.request import urlretrieve
import pickle
import random


def load_data():
    # training set, batches 1-4
    if not os.path.exists(os.path.join(os.getcwd(), "data")):
        os.makedirs(os.path.join(os.getcwd(), "data"))

        
    dataset_dir = os.path.join(os.getcwd(), "data")
    
    if not os.path.exists(os.path.join(dataset_dir, "cifar-10-batches-py")):
        print("Downloading data...")
        urlretrieve("http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz", os.path.join(dataset_dir, "cifar-10-python.tar.gz"))
        tar = tarfile.open(os.path.join(dataset_dir, "cifar-10-python.tar.gz"))
        tar.extractall(dataset_dir)
        tar.close()
        
    train_X = np.zeros((40000, 3, 32, 32), dtype="float32")
    train_y = np.zeros((40000, 1), dtype="ubyte").flatten()
    n_samples = 10000  # aantal samples per batch
    dataset_dir = os.path.join(dataset_dir,"cifar-10-batches-py")
    for i in range(0,4):
        f = open(os.path.join(dataset_dir, "data_batch_"+str(i+1)), "rb")
        cifar_batch = pickle.load(f,encoding="latin1")
        f.close()
        train_X[i*n_samples:(i+1)*n_samples] = (cifar_batch['data'].reshape(-1, 3, 32, 32) / 255.).astype("float32")
        train_y[i*n_samples:(i+1)*n_samples] = np.array(cifar_batch['labels'], dtype='ubyte')

    # validation set, batch 5
    f = open(os.path.join(dataset_dir, "data_batch_5"), "rb")
    cifar_batch_5 = pickle.load(f,encoding="latin1")
    f.close()
    val_X = (cifar_batch_5['data'].reshape(-1, 3, 32, 32) / 255.).astype("float32")
    val_y = np.array(cifar_batch_5['labels'], dtype='ubyte')

    # labels
    f = open(os.path.join(dataset_dir, "batches.meta"), "rb")
    cifar_dict = pickle.load(f,encoding="latin1")
    label_to_names = {k:v for k, v in zip(range(10), cifar_dict['label_names'])}
    f.close()

    # test set
    f = open(os.path.join(dataset_dir, "test_batch"), "rb")
    cifar_test = pickle.load(f,encoding="latin1")
    f.close()
    test_X = (cifar_test['data'].reshape(-1, 3, 32, 32) / 255.).astype("float32")
    test_y = np.array(cifar_test['labels'], dtype='ubyte')


    print("training set size: data = {}, labels = {}".format(train_X.shape, train_y.shape))
    print("validation set size: data = {}, labels = {}".format(val_X.shape, val_y.shape))
    print("test set size: data = {}, labels = {}".format(test_X.shape, test_y.shape))

    return train_X, train_y, val_X, val_y, test_X, test_y, label_to_names


### Preprocessing
Bij CIFAR10 is er niet veel preprocessing nodig. Normalisatie van de data is vaak een goed idee, vantevoren berekenen we de gemiddelde pixelwaarde en bij het batchgewijs trainen normaliseren we de data aan de hand van die waarde. Het is een goed idee om deze mean in een pickle bestand op te slaan, en die dan in te laden bij het opstarten zodat voor predicten niet de hele dataset nodig is. 

In [6]:
train_X, train_y, val_X, val_y, test_X, test_y, label_to_names = load_data()

# Conv nets trainen duurt erg lang op CPU, dus we gebruiken maar een klein deel
# van de data nu, als er tijd over is kan je proberen je netwerk op de volledige set te runnen
train_X = train_X[:10000]
train_y = train_y[:10000]

def calc_mean_std(X):
    mean = np.mean(X)
    std = np.std(X)
    return mean, std

def normalize(data, mean, std):
    return (data-mean)/std

#De data van train_X is genoeg om de mean en std van de hele set nauwkeurig te benaderen
mean,std = calc_mean_std(train_X)
test_X = normalize(test_X,mean,std)
val_X = normalize(val_X,mean,std)
train_X = normalize(train_X ,mean,std)


training set size: data = (40000, 3, 32, 32), labels = (40000,)
validation set size: data = (10000, 3, 32, 32), labels = (10000,)
test set size: data = (10000, 3, 32, 32), labels = (10000,)


# Definieer model
De stap om van een MLP naar een CNN te gaan is niet zo groot, in plaats van een geflattened image gebruiken we nu de originele images als input, en passen we ons model aan om convolutie en pooling layers toe te voegen.

In [ ]:
from keras.models import Model
from keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPooling2D, Input
from sklearn.metrics import classification_report
def conv_net():
    # We definieren de input van het netwerk als de shape van de input,
    # minus de dimensie van het aantal plaatjes, uiteindelijk dus (3, 32, 32).
    input = Input(shape=train_X.shape[1:])
    
    # Eerste convolutielaag
    # Padding valid betekent dat we enkel volledige convoluties gebruiken, zonder padding
    # Data format channels_first betekent dat de channels eerst komen, en dan pas de size van ons plaatje
    # Dus (3, 32, 32) in plaats van (32, 32, 3)
    conv = Conv2D(filters=16, kernel_size=(3,3), padding='valid',
                  data_format='channels_first', activation='relu')(input)
    
    # Nog een convolutielaag, dit keer met stride=2 om de inputsize te verkleinen
    conv = Conv2D(filters=32, kernel_size=(3,3), padding='valid',
                  data_format='channels_first', activation='relu', strides=(2, 2))(conv)
    
    #Voeg een flatten laag toe, om te schakelen naar de dense layer
    flatten = Flatten()(conv)
   
    # De softmax laag voor de probabilities 
    output_layer = Dense(units=nr_classes, activation='softmax')(flatten)
    
    model = Model(inputs=input, outputs=output_layer)
    
    # Het model moet nog gecompiled worden en loss+learning functie gespecificeerd worden
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model


model = conv_net()

model.fit(x=train_X, y=train_y, batch_size=50, epochs=10, validation_data=(val_X, val_y), verbose=2)
predictions = np.array(model.predict(test_X, batch_size=100))
test_y = np.array(test_y, dtype=np.int32)
#Pak de hoogste prediction
predictions = np.argmax(predictions, axis=1)

#Print resultaten
print("Accuracy = {}".format(np.sum(predictions == test_y) / float(len(predictions))))
print(classification_report(test_y, predictions, target_names=list(label_to_names.values())))

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3, 32, 32)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 30, 30)        448       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 14, 14)        4640      
_________________________________________________________________
flatten_1 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                62730     
Total params: 67,818
Trainable params: 67,818
Non-trainable params: 0
_________________________________________________________________


## Opdracht
Wat we graag willen is deze notebook uitgewerkt in een package met het cookiecutter template. We willen dan graag een splitsing tussen het trainingsdeel en het scoring deel. 

Het trainingsdeel levert een model en eventuele metadata op (opgeslagen op disk).  
Het scoringsdeel gebruikt het model om de testset te predicten.  
runnen met: python &lt;filename&gt; 'scoring' of python &lt;filename&gt; 'training'

    
    